In [18]:
#a)
import pandas as pd
kategori_data = pd.read_csv("compas-scores-two-years.csv")
data = data[data["days_b_screening_arrest"] <= 30]
data = data[data["days_b_screening_arrest"] >= -30]
#display(data.columns)
print(data["race"].unique())

['Other' 'African-American' 'Caucasian' 'Hispanic' 'Asian'
 'Native American']


In [15]:
#b)
def finn_sannsynligheter(data):
    #Finner de som ble klassifisert som høy eller lav risiko:
    High_risk = data[data["score_text"].isin(["Medium","High"])]
    Low_risk = data[data["score_text"] == "Low"]

    #Finner hvor bra COMPAS gjorde det:
    Falsk_positiv = High_risk[High_risk["two_year_recid"] == 0]
    Sann_positiv = High_risk[High_risk["two_year_recid"] >= 1]
    Falsk_negativ = Low_risk[Low_risk["two_year_recid"] >= 1]
    Sann_negativ = Low_risk[Low_risk["two_year_recid"] == 0]

    #Finner estimerte sannsynlihgetsverdier:
    p_Falsk_positiv = Falsk_positiv.size/High_risk.size
    p_Sann_positiv = Sann_positiv.size/High_risk.size
    p_Falsk_negativ = Falsk_negativ.size/Low_risk.size
    p_Sann_negativ = Sann_negativ.size/Low_risk.size
    return p_Falsk_positiv, p_Sann_positiv, p_Falsk_negativ, p_Sann_negativ

def print_sannsynligheter(data):
    p_Falsk_positiv, p_Sann_positiv, p_Falsk_negativ, p_Sann_negativ = finn_sannsynligheter(data)
    #Printer sannsynlighetsverdiene:
    print("falsk positiv: ",p_Falsk_positiv)
    print("sann positiv: ",p_Sann_positiv)
    print("falsk negativ: ",p_Falsk_negativ)
    print("sann negativ: ",p_Sann_negativ)
    print()

#Finner de forskjellige typene raser:
print("Hele befolkning:")
print_sannsynligheter(data)

for kjønn in ["Male","Female"]:
    nydata = data[data["sex"] == kjønn]
    print("for ",kjønn,":")
    print_sannsynligheter(nydata)

for rase in ["Caucasian","African-American"]:
    nydata = data[data["race"] == rase]
    print("for ",rase,":")
    print_sannsynligheter(nydata)


Hele befolkning:
falsk positiv:  0.37004725554343876
sann positiv:  0.6299527444565612
falsk negativ:  0.31452791581408945
sann negativ:  0.6854720841859105

for  Male :
falsk positiv:  0.3463736263736264
sann positiv:  0.6536263736263737
falsk negativ:  0.33394562821454815
sann negativ:  0.6660543717854519

for  Female :
falsk positiv:  0.4831932773109244
sann positiv:  0.5168067226890757
falsk negativ:  0.23891273247496422
sann negativ:  0.7610872675250357

for  Caucasian :
falsk positiv:  0.4051724137931034
sann positiv:  0.5948275862068966
falsk negativ:  0.2899786780383795
sann negativ:  0.7100213219616205

for  African-American :
falsk positiv:  0.35046473482777474
sann positiv:  0.6495352651722253
falsk negativ:  0.3514115898959881
sann negativ:  0.6485884101040119



In [22]:
def p_misklassifisert(key,keys,category,data):
    """
    Gir sannsynligheten for at man har blit misklassifisert som høy risiko, 
    gitt at man er i en viss kategori (rase, kjønn, etc)

    f: står for feil
    k: står for at det er den spesifikke kategorien.
    """
    #Finner de som ble klassifisert som høy eller lav risiko:
    data = data[data[category].isin(keys)]
    High_risk = data[data["score_text"].isin(["Medium","High"])]

    #Finner hvor bra COMPAS gjorde det:
    Falsk_positiv = High_risk[High_risk["two_year_recid"] == 0]
    #Falsk_negativ = Low_risk[Low_risk["two_year_recid"] >= 1]

    #P(misklassifisert):
    p_f=(Falsk_positiv.size)/High_risk.size

    #P(misklassifisert|man er i kategorien)
    p_f_k = (Falsk_positiv[Falsk_positiv[category] == key].size
             /High_risk[High_risk[category] == key].size)
    
    #P(man er i kategorien)
    p_k = High_risk[High_risk[category] == key].size/High_risk.size
    
    #Her bruker jeg bayes setning:
    #P(man er i kategorien| man er misklassifisert)
    p_k_f = (p_f_k*p_k)/p_f

    return p_k_f

print(p_misklassifisert("African-American",["African-American","Caucasian"],"race",data))
            

0.6944745395449621
